In [ ]:
## Satellite Salinity files conversion to monthly files

Orginal daily files (from https://doi.org/10.20350%2FdigitalCSIC%2F12620) 

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

from glob import glob
import os.path


In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(dashboard_address=':8188',n_workers=16, threads_per_worker=1)
# cluster = LocalCluster(dashboard_address=':8181',n_workers=8, threads_per_worker=1)
client = Client(cluster)

http://127.0.0.1:8188/status

In [4]:
cluster.close()
client.close()

## Open files

In [4]:
dataDir='/home/ahaddon/data/SSS/09D/'
years=range(2011,2020)
from os import listdir
files={y: sorted(f'{dataDir}{y}/{f}' for f in listdir(f'{dataDir}{y}')) for y in years}

In [5]:
files[2011][:3]

['/home/ahaddon/data/SSS/09D/2011/BEC-L3-SSS-ARCTIC-025km-20110101_20110109-301.nc',
 '/home/ahaddon/data/SSS/09D/2011/BEC-L3-SSS-ARCTIC-025km-20110102_20110110-301.nc',
 '/home/ahaddon/data/SSS/09D/2011/BEC-L3-SSS-ARCTIC-025km-20110103_20110111-301.nc']

In [5]:
xr.open_mfdataset(files[2011][:10], parallel=True)

<xarray.Dataset>
Dimensions:      (time: 10, y: 432, x: 432)
Coordinates:
  * time         (time) datetime64[ns] 2011-01-05T22:05:30 ... 2011-01-14T12:...
  * y            (y) float32 -5.388e+03 -5.362e+03 ... 5.362e+03 5.388e+03
  * x            (x) float32 -5.388e+03 -5.362e+03 ... 5.362e+03 5.388e+03
    lat          (y, x) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lon          (y, x) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
Data variables:
    crs          (time) int32 -2147483647 -2147483647 ... -2147483647
    sss          (time, y, x) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    sss_error    (time, y, x) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    sss_anomaly  (time, y, x) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    sss_flag     (time, y, x) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes: (12/48)
    title:                     Arctic Sea Surface Salinity L3 map
    institution:               Barcelona Expert Center (BEC), ICM-CSIC, Barce...
    institution_url:           http://bec.icm.csic.es
    summary:                   This dataset contains 9-day Level-3 Arctic sea...
    comment:                   These data were produced at BEC as part of the...
    source:                    SMOS L1B version 6.21
    ...                        ...
    sensor:                    SMOS/MIRAS
    spatial_resolution:        35km at nadir
    license:                   This product is distributed under Creative Com...
    license_url:               https://creativecommons.org/licenses/by/4.0/
    copyright:                 If this data is used for publication, the foll...
    doi:                       10.20350/digitalCSIC/12620

## Join files into monthly files

In [4]:
dataDir='/home/ahaddon/data/SSS/09D/'
saveDir='/home/ahaddon/data/SSS/'
years=range(2011,2020)

for y in years:
    print('year '+ str(y))
    for m in range (1,13):
        print('month '+ str(m), end='')
        # check if file exists
        sf=f'{saveDir}BEC-L3-SSS-ARCTIC-025km-{y}-{m:02}.nc'
        if os.path.isfile(sf): continue
        
        if m>1: files=sorted(glob(f'{dataDir}{y}/BEC-L3-SSS-ARCTIC-025km-{y}{m-1:02}*.nc'))[-4:]
        else: files=sorted(glob(f'{dataDir}{y-1}/BEC-L3-SSS-ARCTIC-025km-{y-1}12*.nc'))[-4:]
        files.extend(sorted(glob(f'{dataDir}{y}/BEC-L3-SSS-ARCTIC-025km-{y}{m:02}*.nc'))[:-4])
        # for f in files: print(f[-20:])
        dset=xr.open_mfdataset(files, parallel=True)
        print('\t saving to '+ sf)
        dset.to_netcdf(sf)
        del dset

year 2011
month 1month 2month 3	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-03.nc
month 4	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-04.nc
month 5	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-05.nc
month 6	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-06.nc
month 7	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-07.nc
month 8	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-08.nc
month 9	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-09.nc
month 10	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-10.nc
month 11	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-11.nc
month 12	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2011-12.nc
year 2012
month 1month 2month 3	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2012-03.nc
month 4	 saving to /home/ahaddon/data/SSS/BEC-L3-SSS-ARCTIC-025km-2012-04.nc
month 5	 saving to /home/